How Forecasting is done at places like Walmart/Amazon where they have Millions of SKUs

Some Custom Functions

In [96]:
# Plotting Function
# Some Basic Packages
import pandas as pd
import numpy as np
import plotly.express as px

def plot_forecasts(data, facet_ncol = 1, id_slice = None):

    df = data.copy()

    ids = df['item_id'].unique()

    if id_slice is not None:
        ids = ids[id_slice]

        _filter = df['item_id'].isin(ids)

        df = df[_filter]

    ret = df \
        .melt(
            id_vars    = ['index', 'item_id'],
            value_vars = ['value', 'pred_ets', 'pred_xgb'],
            value_name = 'val'
        ) \
        .pipe(
            func           = px.line,
            x              = 'index',
            y              = 'val',
            color          = 'variable',
            facet_col      = "item_id",
            facet_col_wrap = facet_ncol,
            render_mode    = 'svg',
            line_shape     = 'spline',
            color_discrete_sequence = ["#0055AA", "#C40003", "#00C19B"],
            template       = "plotly_dark"
        ) \
        .update_yaxes(matches=None) \
        .update_layout(showlegend=True, font = dict(size=8)) \
        .update_traces(line = dict(width=0.7))

    return ret

# Making features from time series data
import pandas as pd
import numpy as np

def make_ts_features(df):

    if ('item_id' in df.columns):
        df = df.drop('item_id', axis=1)

    df["date_num"] = df.index.astype('int') / 10**9
    df["date_month"] = df.index.month
    df['date_month_lbl'] = df.index.month_name()
    df['date_month_lbl'] = pd.Categorical(
        df['date_month_lbl'],
        categories=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', ' November', 'December']
    )


    df['date_wday'] = df.index.dayofweek + 1
    df['date_wday_lbl'] = df.index.day_name()
    df['date_wday_lbl'] = pd.Categorical(
        df['date_wday_lbl'],
        categories=['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    )
    df['weekend'] = np.where(df.index.dayofweek <= 5, 0, 1)

    df = pd.get_dummies(df)

    return df

In [9]:
# Read Data
df = pd.read_csv('/content/walmart_item_sales.csv')

In [20]:
df.head()

,item_id,value,date
0,FOODS_3_090,1046,2011-01-29
1,FOODS_3_090,1036,2011-01-30
2,FOODS_3_090,673,2011-01-31
3,FOODS_3_090,642,2011-02-01
4,FOODS_3_090,531,2011-02-02


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193303 entries, 0 to 193302
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   item_id  193303 non-null  category      
 1   value    193303 non-null  int64         
 2   date     193303 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 3.1 MB


In [21]:
# Reformat the Data types
df['item_id'] = pd.Categorical(df['item_id'])

In [14]:
df['date'] = pd.to_datetime(df['date'])

In [43]:
# Visualize-- Pick
# First 12
# _filter = df['item_id'].isin(df['item_id'].unique()[:12])
# Last 12
_filter = df['item_id'].isin(df['item_id'].unique()[-12:])

In [44]:
_filter

0         False
1         False
2         False
3         False
4         False
          ...  
193298     True
193299     True
193300     True
193301     True
193302     True
Name: item_id, Length: 193303, dtype: bool

In [45]:
# Just making the problem smaller for now
df_filtered = df[_filter]

In [46]:
df_filtered \
      .pipe(
          func = px.line,
          x = 'date',
          y = 'value',
          color = 'item_id',
          facet_col = "item_id",
          facet_col_wrap = 3,
          render_mode = 'svg',
          line_shape = 'spline',
          template = 'plotly_dark'
      )\
      .update_yaxes(matches = None) \
      .update_layout(showlegend = False, font = dict(size=8)) \
      .update_traces(line = dict(width=0.7))

In [48]:
# Tip 1: Try 1 Time Series
n = 0

In [49]:
_filter = df['item_id'].isin(
    # notice the extra brackets
    # isin only works for a list
    [df['item_id'].unique()[n]]
)

In [50]:
df_sample = df[_filter] \
        .set_index('date')

In [54]:
# Cool, just one item!
# Notice Date is the index
# looking more like how we like our time series!
df_sample

,item_id,value
date,,
2011-01-29,FOODS_3_090,1046
2011-01-30,FOODS_3_090,1036
2011-01-31,FOODS_3_090,673
2011-02-01,FOODS_3_090,642
2011-02-02,FOODS_3_090,531
...,...,...
2016-04-20,FOODS_3_090,425
2016-04-21,FOODS_3_090,422
2016-04-22,FOODS_3_090,629


In [56]:
df_sample.index.freq = 'd'
df_sample.index

DatetimeIndex(['2011-01-29', '2011-01-30', '2011-01-31', '2011-02-01',
               '2011-02-02', '2011-02-03', '2011-02-04', '2011-02-05',
               '2011-02-06', '2011-02-07',
               ...
               '2016-04-15', '2016-04-16', '2016-04-17', '2016-04-18',
               '2016-04-19', '2016-04-20', '2016-04-21', '2016-04-22',
               '2016-04-23', '2016-04-24'],
              dtype='datetime64[ns]', name='date', length=1913, freq='D')

In [57]:
df_sample['value'] = df_sample['value'].astype('float')

In [58]:
df_sample['value']

date
2011-01-29    1046.0
2011-01-30    1036.0
2011-01-31     673.0
2011-02-01     642.0
2011-02-02     531.0
               ...  
2016-04-20     425.0
2016-04-21     422.0
2016-04-22     629.0
2016-04-23     793.0
2016-04-24     592.0
Freq: D, Name: value, Length: 1913, dtype: float64

In [60]:
!pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 14.8 MB/s eta 0:00:00


In [61]:
# Split the time series into Train/Test
# Lets use some time series specialized splitting function
from sktime.forecasting.model_selection import temporal_train_test_split

In [62]:
df_train, df_test = temporal_train_test_split(
    df_sample,
    test_size = 90
)

In [63]:
df_train.index

DatetimeIndex(['2011-01-29', '2011-01-30', '2011-01-31', '2011-02-01',
               '2011-02-02', '2011-02-03', '2011-02-04', '2011-02-05',
               '2011-02-06', '2011-02-07',
               ...
               '2016-01-16', '2016-01-17', '2016-01-18', '2016-01-19',
               '2016-01-20', '2016-01-21', '2016-01-22', '2016-01-23',
               '2016-01-24', '2016-01-25'],
              dtype='datetime64[ns]', name='date', length=1823, freq='D')

In [64]:
df_test.index

DatetimeIndex(['2016-01-26', '2016-01-27', '2016-01-28', '2016-01-29',
               '2016-01-30', '2016-01-31', '2016-02-01', '2016-02-02',
               '2016-02-03', '2016-02-04', '2016-02-05', '2016-02-06',
               '2016-02-07', '2016-02-08', '2016-02-09', '2016-02-10',
               '2016-02-11', '2016-02-12', '2016-02-13', '2016-02-14',
               '2016-02-15', '2016-02-16', '2016-02-17', '2016-02-18',
               '2016-02-19', '2016-02-20', '2016-02-21', '2016-02-22',
               '2016-02-23', '2016-02-24', '2016-02-25', '2016-02-26',
               '2016-02-27', '2016-02-28', '2016-02-29', '2016-03-01',
               '2016-03-02', '2016-03-03', '2016-03-04', '2016-03-05',
               '2016-03-06', '2016-03-07', '2016-03-08', '2016-03-09',
               '2016-03-10', '2016-03-11', '2016-03-12', '2016-03-13',
               '2016-03-14', '2016-03-15', '2016-03-16', '2016-03-17',
               '2016-03-18', '2016-03-19', '2016-03-20', '2016-03-21',
      

In [66]:
# Modeling
# ETS -- Error Trend Season -- Exponential smoothing family
# Quite similar to ARIMA

# Forecasting Horizon
# start at 1
# weird that the function is return this way!
fh = np.arange(1,91)
fh

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90])

In [69]:
# Similar to AutoArima-- We have AutoETS
# sp - seasonal period
from sktime.forecasting.ets import AutoETS

In [72]:
forecaster_ets = AutoETS(
    auto = True,
    n_jobs = -1,
    sp = 7,
    additive_only = True
)

In [73]:
# Fit the data
forecaster_ets.fit(df_train['value'])

AutoETS(additive_only=True, auto=True, n_jobs=-1, sp=7)

In [74]:
# Make forecast
y_pred_ets = forecaster_ets.predict(fh)
y_pred_ets.name = 'pred_ets'
y_pred_ets

2016-01-26    396.993266
2016-01-27    432.953294
2016-01-28    449.814130
2016-01-29    526.559165
2016-01-30    639.666603
                 ...    
2016-04-20    432.953294
2016-04-21    449.814130
2016-04-22    526.559165
2016-04-23    639.666603
2016-04-24    573.359432
Freq: D, Name: pred_ets, Length: 90, dtype: float64

In [76]:
# Visualize
# Combine the two series
results_df = pd.concat([df_sample, y_pred_ets], axis = 1) \
              .reset_index()

In [80]:
results_df

,index,item_id,value,pred_ets
0,2011-01-29,FOODS_3_090,1046.0,NaN
1,2011-01-30,FOODS_3_090,1036.0,NaN
2,2011-01-31,FOODS_3_090,673.0,NaN
3,2011-02-01,FOODS_3_090,642.0,NaN
4,2011-02-02,FOODS_3_090,531.0,NaN
...,...,...,...,...
1908,2016-04-20,FOODS_3_090,425.0,432.953294
1909,2016-04-21,FOODS_3_090,422.0,449.814130
1910,2016-04-22,FOODS_3_090,629.0,526.559165
1911,2016-04-23,FOODS_3_090,793.0,639.666603


In [81]:
# Graph
results_df \
      .pipe(
          func = px.line,
          x = 'index',
          y = ['value', 'pred_ets'],
          template = "plotly_dark",
          render_mode = 'svg',
          line_shape = 'spline'
      )

In [79]:
# Performance/Evaluation - ETS Model
import sklearn.metrics as metrics
import sktime.performance_metrics.forecasting as ts_metrics

In [83]:
assess_df = pd.concat([df_test, y_pred_ets], axis=1)

metrics.r2_score(
    y_true=assess_df['value'],
    y_pred=assess_df['pred_ets']
)

0.5354056103722031

In [84]:
ts_metrics.mean_absolute_error(
    y_true=assess_df['value'],
    y_pred=assess_df['pred_ets']
)

80.72484752906409

In [85]:
# Future Forecast -- ETS model
# Now that we are satified with the model
# Use the full dataset -- train and test to make the final model
forecaster_ets.update(
    y = df_sample['value']
)

/usr/local/lib/python3.10/dist-packages/sktime/forecasting/base/_base.py:2055: UserWarning:




AutoETS(additive_only=True, auto=True, n_jobs=-1, sp=7)

In [86]:
# Can forecast different length than the test set but
# we will do the same
future_pred_ets = forecaster_ets.predict(fh)\
                  .rename("pred_ets")

In [87]:
future_pred_ets

2016-04-25    411.075168
2016-04-26    398.173715
2016-04-27    450.679165
2016-04-28    474.596064
2016-04-29    604.788820
                 ...    
2016-07-19    398.173715
2016-07-20    450.679165
2016-07-21    474.596064
2016-07-22    604.788820
2016-07-23    720.883690
Freq: D, Name: pred_ets, Length: 90, dtype: float64

In [92]:
# XGBoost Model
# These Machine learning models
# need an extra step
# Take time series data
# and extract features

In [95]:
!pip install forecasting

ERROR: Could not find a version that satisfies the requirement forecasting (from versions: none)
ERROR: No matching distribution found for forecasting


In [99]:
df_sample

# Feature Engineering -----

df_sample_features = make_ts_features(df_sample)

df_train, df_test = temporal_train_test_split(
    df_sample_features,
    test_size=90
)


In [101]:
df_sample_features.head()

,value,date_num,date_month,date_wday,weekend,date_month_lbl_January,date_month_lbl_February,date_month_lbl_March,date_month_lbl_April,date_month_lbl_May,...,date_month_lbl_October,date_month_lbl_ November,date_month_lbl_December,date_wday_lbl_Sunday,date_wday_lbl_Monday,date_wday_lbl_Tuesday,date_wday_lbl_Wednesday,date_wday_lbl_Thursday,date_wday_lbl_Friday,date_wday_lbl_Saturday
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,1046.0,1.296259e+09,1,6,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2011-01-30,1036.0,1.296346e+09,1,7,1,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2011-01-31,673.0,1.296432e+09,1,1,0,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2011-02-01,642.0,1.296518e+09,2,2,0,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2011-02-02,531.0,1.296605e+09,2,3,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [89]:
# XGboost
from xgboost import XGBRegressor

In [90]:
forecaster_xgb = XGBRegressor()

In [98]:
forecaster_xgb.fit(
    X = df_train.drop('value', axis=1),
    y = df_train['value']
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [102]:
y_pred_xgb = forecaster_xgb.predict(df_test.drop('value', axis=1))

In [103]:
# Previous Steps array. - its scikit's style
# lets convert to pandas series
y_pred_xgb = pd.Series(y_pred_xgb, index=df_test.index) \
    .rename("pred_xgb")


In [104]:
assess_df = pd.concat([df_test.value, y_pred_xgb], axis=1) \
    .reset_index()

In [109]:
metrics.r2_score(
    y_true=assess_df['value'],
    y_pred=assess_df['pred_xgb']
)

# Low score is bad

-0.70835286552778

In [110]:
ts_metrics.mean_absolute_error(
    y_true=assess_df['value'],
    y_pred=assess_df['pred_xgb']
)

153.46603071424695

In [111]:
# FUTURE FORECAST ----

future_series = pd.date_range(
    start = df_sample_features.index[-1] + pd.DateOffset(days=1),
    periods = 90
)

X_future = pd.DataFrame(dict(date = future_series)) \
    .set_index('date') \
    .pipe(
        make_ts_features
    )

forecaster_xgb.fit(df_sample_features.drop('value', axis=1), df_sample_features['value'])

future_pred_xgb = forecaster_xgb.predict(X_future)

future_pred_xgb = pd.Series(future_pred_xgb, index=future_series) \
    .rename("pred_xgb")

future_pred_xgb

2016-04-25    455.121246
2016-04-26    407.748322
2016-04-27    414.830811
2016-04-28    434.656403
2016-04-29    643.463989
                 ...    
2016-07-19    408.610352
2016-07-20    506.604462
2016-07-21    651.294434
2016-07-22    679.475037
2016-07-23    865.836060
Freq: D, Name: pred_xgb, Length: 90, dtype: float32

In [112]:
# FUTURE VISUALIZATION - ETS & XGBOOST ---- together
pd.concat([df_sample, future_pred_ets, future_pred_xgb], axis = 1) \
    .assign(item_id = lambda x: x['item_id'].ffill()) \
    .reset_index() \
    .melt(
        id_vars    = ['index', 'item_id'],
        value_vars = ['value', 'pred_ets', 'pred_xgb'],
        value_name = 'val'
    ) \
    .pipe(
        func           = px.line,
        x              = 'index',
        y              = 'val',
        color          = 'variable',
        facet_col      = "item_id",
        render_mode    = 'svg',
        line_shape     = 'spline',
        color_discrete_sequence = ["#0055AA", "#C40003", "#00C19B"],
        template       = "plotly_dark"
    ) \
    .update_layout(showlegend=True, font = dict(size=8)) \
    .update_traces(line = dict(width=0.7))

In [121]:
# Outside the scope of the class
# What if I want to loop through all 100 items


import logging
import os

import pandas as pd
import numpy as np
import plotly.express as px

from sktime.forecasting.model_selection import temporal_train_test_split

import sklearn.metrics as metrics
import sktime.performance_metrics.forecasting as ts_metrics

from xgboost import XGBRegressor
from sktime.forecasting.ets import AutoETS


def run_forecasts(data, verbose=True, log_errors=True, id_slice=None):

    error_file = "error_logs.txt"
    if log_errors:
        if os.path.exists(error_file):
            os.remove(error_file)

    df = data.copy()

    # Prep Data
    df['item_id'] = pd.Categorical(df['item_id'])
    df['date'] = pd.to_datetime(df['date'])

    # Make iterables
    ids = df['item_id'].unique()

    if id_slice is not None:
        ids = ids[id_slice]

    n_obs = len(ids)

    li = []

    for i, id in enumerate(ids):

        try:

            if verbose: print(f"[{i+1}/{n_obs}] {id}")

            # Select Single Time Series
            df_sample = select_time_series(df, n=i)

            # Split Time Series ----
            df_train, df_test = temporal_train_test_split(
                df_sample,
                test_size=90
            )

            # ETS Model ----
            ets_results = ets_train_test(df_train, df_test)

            if verbose:
                print(f"  [SUCCESS] ETS Model R-squared: {np.round(ets_results['score'],3)}")

            # XGBoost Model ----
            xgb_results = xgb_train_test(df_train, df_test)

            if verbose:
                print(f"  [SUCCESS] XGB Model R-squared: {np.round(xgb_results['score'],3)}")

            # Compare ----
            if (ets_results['score'] > xgb_results['score']):

                if verbose:
                    print(f"  [MODEL SELECTED] ETS")

                forecaster_ets = ets_results['forecaster']

                forecaster_ets.update(
                    y = df_sample['value']
                )

                n_obs = df_test.shape[0]
                fh = np.arange(1, n_obs+1)

                future_pred = forecaster_ets.predict(fh) \
                    .rename("pred_ets")

            else:
                if verbose:
                    print(f"  [MODEL SELECTED] XGBOOST")

                forecaster_xgb = xgb_results['forecaster']

                df_sample_features = make_ts_features(df_sample)

                future_series = pd.date_range(
                    start = df_sample_features.index[-1] + pd.DateOffset(days=1),
                    periods = 90
                )

                X_future = pd.DataFrame(dict(date = future_series)) \
                    .set_index('date') \
                    .pipe(
                        make_ts_features
                    )

                forecaster_xgb.fit(df_sample_features.drop('value', axis=1), df_sample_features['value'])

                future_pred_xgb = forecaster_xgb.predict(X_future)

                future_pred = pd.Series(future_pred_xgb, index=future_series) \
                    .rename("pred_xgb")



            # Collect Results

            predictions_df = pd.concat([df_sample, future_pred], axis = 1) \
                .assign(item_id = lambda x: x['item_id'].ffill()) \
                .reset_index()

            # print(predictions_df)

            li = li + [predictions_df]

        except Exception as Argument:

            msg = f"An error occurred in {i}: {id}."

            if verbose:
                logging.exception(msg)

            if log_errors:
                f = open(error_file, "a")
                f.write(msg)
                f.write("\n")
                f.write(str(Argument))
                f.write("\n\n")
                f.close()

        print(id)

    ret = pd.concat(li, axis=0)

    return ret


# FUNCTIONS ----

def select_time_series(data, n = 0):

    df = data.copy()

    # Select Time Series ----
    _filter = df['item_id'].isin(
        # Make sure it's a list
        [df['item_id'].unique()[n]]
    )

    df_sample = df[_filter] \
        .set_index('date')
    df_sample

    df_sample.index.freq = 'd'
    df_sample.index

    df_sample['value'] = df_sample['value'].astype('float')

    return df_sample



def ets_train_test(df_train, df_test):

    # ETS MODEL ----

    # Forecast Horizon ----
    n_obs = df_test.shape[0]
    fh = np.arange(1, n_obs+1)

    # Forecasting - ETS Model
    forecaster_ets = AutoETS(
        auto=True,
        n_jobs=-1,
        sp = 7,
        additive_only=True
    )

    forecaster_ets.fit(y = df_train['value'])

    y_pred_ets = forecaster_ets.predict(fh)
    y_pred_ets.name = 'pred_ets'
    y_pred_ets

    # Metrics
    assess_df = pd.concat([df_test, y_pred_ets], axis=1)

    score = metrics.r2_score(
        y_true=assess_df['value'],
        y_pred=assess_df['pred_ets']
    )

    ret = dict(
        score = score,
        forecaster = forecaster_ets
    )

    return ret

def xgb_train_test(df_train, df_test):

    # XGBOOST MODEL ----

    # Feature Engineering
    df_train = make_ts_features(df_train)
    df_test = make_ts_features(df_test)

    # Xgboost Model
    forecaster_xgb = XGBRegressor()

    forecaster_xgb.fit(
        X = df_train.drop('value', axis=1),
        y = df_train['value']
    )

    # Performance
    y_pred_xgb = forecaster_xgb.predict(df_test.drop('value', axis=1))

    y_pred_xgb = pd.Series(y_pred_xgb, index=df_test.index) \
        .rename("pred_xgb")

    assess_df = pd.concat([df_test.value, y_pred_xgb], axis=1) \
        .reset_index()

    score = metrics.r2_score(
        y_true=assess_df['value'],
        y_pred=assess_df['pred_xgb']
    )

    # Return
    ret = dict(
        score = score,
        forecaster = forecaster_xgb
    )

    return ret



In [115]:
# Lets reread the data
df = pd.read_csv('/content/walmart_item_sales.csv')

In [122]:
# Run Automation
best_forecasts_df = run_forecasts(
    df,
    id_slice=None
)

[1/102] FOODS_3_090
  [SUCCESS] ETS Model R-squared: 0.535
  [SUCCESS] XGB Model R-squared: -0.708
  [MODEL SELECTED] ETS


/usr/local/lib/python3.10/dist-packages/sktime/forecasting/base/_base.py:2055: UserWarning:




FOODS_3_090
[2/90] FOODS_3_586
  [SUCCESS] ETS Model R-squared: 0.294
  [SUCCESS] XGB Model R-squared: -0.293
  [MODEL SELECTED] ETS


/usr/local/lib/python3.10/dist-packages/sktime/forecasting/base/_base.py:2055: UserWarning:




FOODS_3_586
[3/90] FOODS_3_252
  [SUCCESS] ETS Model R-squared: -0.469
  [SUCCESS] XGB Model R-squared: 0.089
  [MODEL SELECTED] XGBOOST
FOODS_3_252
[4/90] FOODS_3_555
  [SUCCESS] ETS Model R-squared: -0.55
  [SUCCESS] XGB Model R-squared: -0.844
  [MODEL SELECTED] ETS


...

FOODS_2_347
[99/90] HOUSEHOLD_1_409
  [SUCCESS] ETS Model R-squared: -0.173
  [SUCCESS] XGB Model R-squared: -0.089
  [MODEL SELECTED] XGBOOST
HOUSEHOLD_1_409
[100/90] FOODS_3_154
  [SUCCESS] ETS Model R-squared: -0.617
  [SUCCESS] XGB Model R-squared: -1.298
  [MODEL SELECTED] ETS


/usr/local/lib/python3.10/dist-packages/sktime/forecasting/base/_base.py:2055: UserWarning:




FOODS_3_154
[101/90] HOBBIES_2_119
  [SUCCESS] ETS Model R-squared: -0.023
  [SUCCESS] XGB Model R-squared: -0.205
  [MODEL SELECTED] ETS


/usr/local/lib/python3.10/dist-packages/sktime/forecasting/base/_base.py:2055: UserWarning:


ERROR:root:An error occurred in 101: HOUSEHOLD_2_101.
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 463, in _process_worker
    r = call_item()
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 589, in __call__
    return [func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 589, in <listcomp>
    return [func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sktime/forecasting/ets.py", line 334, in _fit
    _forecaster = _ETSModel(
  File "/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/exponential_smo

HOBBIES_2_119
[102/90] HOUSEHOLD_2_101
HOUSEHOLD_2_101


In [124]:
# Plot Automation

plot_forecasts(
    best_forecasts_df,
    facet_ncol = 3,
    id_slice = np.arange(0,12)
)
